# Troubleshooting online endpoints deployment and scoring

SRC: ['How to troubleshoot online endpoints'](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints)

## Get container logs

SRC: ['How to troubleshoot online endpoints'](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints)

If you are using the AML Inference Server, you can increase the logging detail by setting the `AZUREML_LOG_LEVEL` environment variable to `debug` or `notset` (full logging). For custom container deployments with alternative inference servers, logging should be configured to stream to `STDOUT`.

## Get deployment logs

After submitting the deployment operation with `az ml online-deployment create`, the CLI polls Azure for updates until the final result is delivered. Include the `--verbose` flag for additional information and `--debug` to see every call in full detail. 

The operation result will include an error message in case of failure. Common error messages and debugging tips are described below. The operation result will available at the deployment's AzureAsyncOperationUri and can be called with the following command:

```bash
DEPLOYMENT_NAME="<YOUR_DEPLOYMENT_NAME>"
ENDPOINT_NAME="<YOUR_ENDPOINT_NAME>"
async=`az ml online-deployment show -n $DEPLOYMENT_NAME --endpoint-name $ENDPOINT_NAME --query properties.AzureAsyncOperationUri -o tsv`
az rest -m get --url $async
```

[Azure Activity Log](/azure/azure-monitor/essentials/activity-log) can be configured to save, track, and query deployment logs.

## Request tracing 

SRC: 
['How to troubleshoot online endpoints'](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints) 


### App Insights

For advanced request tracing and log consumption, see [How to Enable App Insights](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-enable-app-insights). App insights can be enabled by setting `app_insights_enabled` in the Online Deployment YAML file. To use app insights with the AML Inference Server, the environment variable `AML_APP_INSIGHTS_KEY` must be provided. 

## Common deployment errors
### ResourceNotReady
SRC: ['How to troubleshoot online endpoints'](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints)

ResourceNotReady errors happen when the liveness or readiness routes are specified incorrectly. If your deployment script is not readily available, check the environment settings with this command:

```bash
VERSION=`az ml online-deployment show -n $DEPLOYMENT_NAME --endpoint-name $ENDPOINT_NAME --query environment -o tsv`
az ml environment show -n CliV2AnonymousEnvironment --version ${VERSION##*/}
``` 

### Upstream Connect Error / Connection Failure
Upstram connect errors happen when invoking an endpoint for a deployment with a dead or incorrect scoring route. Your error message may look like this `upstream connect error or disconnect/reset before headers. reset reason: connection failure`. Online deployments with incorrect scoring routes do not necessarily fail to deploy. 

## HTTP Errors 
### 104 (Connection Aborted / ConnectionResetError)

Deployment not online and/or traffic is set to 0%. 
